In [1]:
# Import libraries

import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [25]:
# Import df

df = pd.read_csv('NY_bikedata_sample.csv', index_col = 0)

C:\Users\walls\AppData\Local\Temp\ipykernel_6636\30057621.py:3: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NY_bikedata_sample.csv', index_col = 0)


In [3]:
df.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,rides_daily,_merge
ride_id,,,,,,,,,,,,,,,,
DA14E2F38543A9FF,classic_bike,2021-12-28 12:17:45.393,2022-01-01 11:07:06.792,Southpoint Park,6566.01,NaN,NaN,40.75370,-73.958650,NaN,NaN,casual,2021-12-28,NaN,1,both
23643AF38D9D91ED,classic_bike,2021-12-31 09:49:35.149,2022-01-01 10:49:15.878,W 47 St & 10 Ave,6824.07,NaN,NaN,40.76270,-73.993010,NaN,NaN,casual,2021-12-31,NaN,19,both
53B738C6594CC191,electric_bike,2021-12-31 23:51:45.918,2022-01-01 00:20:15.060,Broadway & W 61 St,7014.12,E 88 St & 1 Ave,7235.13,40.77003,-73.981970,40.778300,-73.948814,casual,2021-12-31,NaN,19,both
2065757132FE97C0,classic_bike,2021-12-31 23:58:39.100,2022-01-01 00:01:02.766,W 47 St & 10 Ave,6824.07,W 49 St & 8 Ave,6747.06,40.76270,-73.993010,40.762270,-73.987885,casual,2021-12-31,NaN,19,both
FD0283A8C2A6FAEF,electric_bike,2021-12-31 23:51:27.961,2022-01-01 00:16:04.623,Wythe Ave & Metropolitan Ave,5348.02,DeKalb Ave & Franklin Ave,4528.01,40.71689,-73.963196,40.690647,-73.957460,member,2021-12-31,NaN,19,both


In [4]:
df.shape

(1000000, 16)

In [5]:
df.dtypes

rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
date                   object
avgTemp               float64
rides_daily             int64
_merge                 object
dtype: object

In [6]:
# Drop _merge column 

df.drop(columns = {'_merge'}, inplace = True)

# Convert to float32 for memory, keep NaNs

df[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'avgTemp']] = (
    df[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'avgTemp']].astype('float32'))

In [7]:
df.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,rides_daily
ride_id,,,,,,,,,,,,,,,
DA14E2F38543A9FF,classic_bike,2021-12-28 12:17:45.393,2022-01-01 11:07:06.792,Southpoint Park,6566.01,NaN,NaN,40.753700,-73.958649,NaN,NaN,casual,2021-12-28,NaN,1
23643AF38D9D91ED,classic_bike,2021-12-31 09:49:35.149,2022-01-01 10:49:15.878,W 47 St & 10 Ave,6824.07,NaN,NaN,40.762699,-73.993011,NaN,NaN,casual,2021-12-31,NaN,19
53B738C6594CC191,electric_bike,2021-12-31 23:51:45.918,2022-01-01 00:20:15.060,Broadway & W 61 St,7014.12,E 88 St & 1 Ave,7235.13,40.770031,-73.981972,40.778301,-73.948814,casual,2021-12-31,NaN,19
2065757132FE97C0,classic_bike,2021-12-31 23:58:39.100,2022-01-01 00:01:02.766,W 47 St & 10 Ave,6824.07,W 49 St & 8 Ave,6747.06,40.762699,-73.993011,40.762272,-73.987885,casual,2021-12-31,NaN,19
FD0283A8C2A6FAEF,electric_bike,2021-12-31 23:51:27.961,2022-01-01 00:16:04.623,Wythe Ave & Metropolitan Ave,5348.02,DeKalb Ave & Franklin Ave,4528.01,40.716888,-73.963196,40.690647,-73.957458,member,2021-12-31,NaN,19


In [8]:
df.shape

(1000000, 15)

In [9]:
df.dtypes

rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float32
start_lng             float32
end_lat               float32
end_lng               float32
member_casual          object
date                   object
avgTemp               float32
rides_daily             int64
dtype: object

#### 2. Grouping data 

In [10]:
df.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'member_casual', 'date', 'avgTemp',
       'rides_daily'],
      dtype='object')

In [11]:
# starting station, ending station, longitutes and latitudes, and the count of trips between those stations

df['trips'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])['trips'].count().reset_index()

In [12]:
df_group.head()

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,40.792328,-73.938301,40.792328,-73.938301,26
1,1 Ave & E 110 St,1 Ave & E 110 St,40.792385,-73.938065,40.792328,-73.938301,1
2,1 Ave & E 110 St,1 Ave & E 110 St,40.792397,-73.938240,40.792328,-73.938301,1
3,1 Ave & E 110 St,1 Ave & E 110 St,40.792400,-73.938164,40.792328,-73.938301,1
4,1 Ave & E 110 St,1 Ave & E 62 St,40.792328,-73.938301,40.761227,-73.960938,1


In [13]:
print(df_group['trips'].sum())
print(df.shape)

997578
(1000000, 16)


In [14]:
# Check for NaN values

df_group.isna().sum()

start_station_name    0
end_station_name      0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
trips                 0
dtype: int64

In [15]:
print(df['start_station_name'].isna().sum())
print(df['end_station_name'].isna().sum())
print(df['start_lat'].isna().sum())
print(df['end_lat'].isna().sum())
print(df['start_lng'].isna().sum())
print(df['end_lng'].isna().sum())

1
2422
0
1289
0
1289


In [16]:
# fill NaN values with "Uknown"

df['start_station_name'] = df['start_station_name'].fillna('Unknown')
df['end_station_name'] = df['end_station_name'].fillna('Unknown')
df['start_lat'] = df['start_lat'].fillna('Unknown')
df['end_lat'] = df['end_lat'].fillna('Unknown')
df['start_lng'] = df['start_lng'].fillna('Unknown')
df['end_lng'] = df['end_lng'].fillna('Unknown')

In [17]:
print(df['start_station_name'].isna().sum())
print(df['end_station_name'].isna().sum())
print(df['start_lat'].isna().sum())
print(df['end_lat'].isna().sum())
print(df['start_lng'].isna().sum())
print(df['end_lng'].isna().sum())

0
0
0
0
0
0


In [18]:
df.shape

(1000000, 16)

#### Notes: 

NaN values in station start and end columns were changed to "uknown" to still allow for analysis of trips taken
Later investigation can try to determine why these station names are unkown.

In [19]:
# regrouping values 

df_group = df.groupby(['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])['trips'].count().reset_index()

In [20]:
print(df_group['trips'].sum())
print(df.shape)

1000000
(1000000, 16)


#### 3. Mapping Trips

In [21]:
# Create KeplerGl instance

map = KeplerGl(height = 700, data={"data_1": df_group})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [22]:
map

C:\Users\walls\AppData\Roaming\Python\Python311\site-packages\IPython\core\formatters.py:367: FormatterWarning: text/html formatter returned invalid type <class 'bytes'> (expected <class 'str'>) for object: KeplerGl(data={'data_1':            start_station_name       end_station_name  start_lat  start_lng  \
0            1 Ave & E 110 St       1 Ave & E 110 St  40.792328 -73.938301   
1            1 Ave & E 110 St       1 Ave & E 110 St  40.792385 -73.938065   
2            1 Ave & E 110 St       1 Ave & E 110 St  40.792397 -73.938240   
3            1 Ave & E 110 St       1 Ave & E 110 St  40.792400 -73.938164   
4            1 Ave & E 110 St        1 Ave & E 62 St  40.792328 -73.938301   
...                       ...                    ...        ...        ...   
394925  Yankee Ferry Terminal                Unknown  40.687119 -74.016655   
394926  Yankee Ferry Terminal     Water St & Main St  40.687065 -74.016754   
394927  Yankee Ferry Terminal  West St & Chambers St  40.687065 -7

KeplerGl(data={'data_1':            start_station_name       end_station_name  start_lat  start_lng  \
0      …

##### Notes: 
I chose to have three layers for the mapping of bike trips. Two of the layers are to show the start of a bike trip and the end of a bike trip. It could be useful to see how far users are going on their bikes for each trip. To help see this more clearly, the third layer is applied which draws a line that connects the start and end points. It actually appears stations with more trips also rarely have long trips. 
To better see stations with a high frequency of trips, a filter was added. The filter starts at 100 trips. We can see the more popular stations and we can also see that some points on the map are no connected. It would be good to see how often bikes are returned to the same station. Perhaps users returning bikes to different stations is a culprit for shortages as well.

#### Exports

In [23]:
# save settings 

config = map.config

In [24]:
# export map

map.save_to_html(file_name='100+ Bike Trips.html', read_only = False, config = config)

Map saved to 100+ Bike Trips.html!


In [26]:
# export data frames

df.to_csv('NY_bikedata2.csv', index=False)
df_group.to_csv('NY_bikedata_trips.csv', index=False)